# Notebook for the final assigment os the course Applied Data Science Capstone

In [33]:
import pandas as pd
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df=pd.read_html(url, header=0)[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned

In [34]:
#Let just the rows with Borough different to Not Assigned
df_wo_Borough = df[df['Borough'] != 'Not assigned']

#Reset index
df_wo_Borough.reset_index(inplace=True)

#Delete created column index
df_wo_Borough.drop(['index'], axis=1, inplace = True)

df_wo_Borough.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [35]:
#Combine same neighborhoods with the same postal code
df_combine = df_wo_Borough.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join)

In [36]:
#Convert the result in a DataFrame
df_combine =  pd.DataFrame(df_combine)

#Reset the index
df_combine.reset_index(inplace = True)
df_combine.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [37]:
#Rename the column
df_combine.rename(columns={'Neighbourhood': 'Neighborhood'}, inplace = True)
df_combine.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [39]:
#Check if it is any row with Neighborhood not assigned
df_combine[df_combine['Neighborhood'] == 'Not assigned']

,Postcode,Borough,Neighborhood
85,M7A,Queen's Park,Not assigned


In [41]:
#Correct rows with not assigned neighborhood
for index, rows in df_combine.iterrows():
    if rows['Neighborhood'] == 'Not assigned':
        print('The neighborhood in the position', index, 'is equal to', rows['Neighborhood'], 'and the Borough is equal to', rows['Borough'])
        df_combine.iloc[index]['Neighborhood']=df_combine.iloc[index]['Borough']
        print('Know the neighborhood is', df_combine.iloc[index]['Neighborhood'])

The neighborhood in the position 85 is equal to Not assigned and the Borough is equal to Queen's Park
Know the neighborhood is Queen's Park


In [42]:
#Check the correction od the neigboorhod
df_combine.iloc[85]

Postcode                 M7A
Borough         Queen's Park
Neighborhood    Queen's Park
Name: 85, dtype: object

In [45]:
#Check the shape of the dataframe
df_combine.shape

(103, 3)